# Juliaで非線型方程式の解の精度保証付き数値計算

非線形方程式
$$
f(x) = 0,\quad f:\mathbb{R}^n \rightarrow \mathbb{R}^n
$$
の解を精度保証することを考える。

今回は、区間解析の標準的な手法となっているR.Krawczykによる解の検証手法を紹介する。

## Krawczyk法

Krawczykの主張は、
$X\in \mathbb{R}^n$を区間ベクトル（候補者集合）、$c = mid(X)$、$R\simeq f'(c)^{-1}$、$E$を単位行列とし、

\begin{equation}
K(X) = c-Rf(c)+(E-Rf'(X))(X-c)
\end{equation}

としたとき、$K(X)\in int(X)$ならば$X$に$f(x)=0$の解が唯一存在する。
というものである。（$int(X)$：$X$の内部）

この主張を成り立たせるためには、$g(x)=x-Rf(x)$に対して、平均値形式と縮小写像原理を適用する必要がある。

## 平均値形式とKrawczyk写像

$X\in \mathbb{R}^n$を区間ベクトルとし、写像$f$に対する簡易ニュートン写像$g:\mathbb{R}^n\rightarrow \mathbb{R}^n$を次で定義する。

$$
g(x)=x-Rf(x)
$$

$R\in \mathbb{R}^{n\times n}$は、$n$次元正方行列として、cにおけるヤコビ行列$J(c)$の逆近似行列$(R\simeq f'(c)^{-1})$とする。

このとき、$R$が正則であるならば、
$$
f(x)=0\Longleftrightarrow g(x)=x
$$
が成り立つ。

そして、$c$が、区間ベクトル$X$から$X$への縮小写像となれば、縮小写像の原理から、$f(x)=0$の真の解$x^*$が$X$内に一意存在することが示せる。しかし、写像$g$の区間$X$における区間拡張$s_{[]}(X)$を考えると、常に$s_{[]}(X)=X-Rf_{[]}(X)\not\subset X$となり、縮小写像の原理を生かすことができない。そこで、期間演算における区間の増大を抑制するための基本手法である平均値形式を導入する。

## 自動微分を利用したヤコビ行列の計算

Kwawczyk法を使う際には、区間拡張$f'(x)$を計算する必要がある。計算の方法として、最も標準的な実装方法は、自動微分を利用することである。

Juliaで自動微分を適用する場合には、ForwardDiffパッケージを利用する。使用例は以下の通りである。

In [2]:
using LinearAlgebra,IntervalArithmetic, ForwardDiff
X = [(0.8..0.9),(-1..(-0.5))]
# mid(X)
# X = rand(2)
f(x, y) = x^2 + y^2 - 1
g(x, y) = x^3 + y^4
h( (x, y) ) = [f(x, y); g(x, y)]
# ForwardDiff.jacobian(g, X::IntervalBox) = ForwardDiff.jacobian(g, X.v)
J = ForwardDiff.jacobian(h,X)

2×2 Array{Interval{Float64},2}:
 [1.59999, 1.80001]  [-2, -1]
 [1.91999, 2.43001]   [-4, -0.5]

In [3]:
h(X)

2-element Array{Interval{Float64},1}:
 [-0.110001, 0.810001]
  [0.574499, 1.72901]

## Krawczyk法の実装

ここから、JuliaにおけるKrawczyk法の実装を行う。実装例は以下の通りである。

In [41]:
using LinearAlgebra,IntervalArithmetic, ForwardDiff
#区間Xを定義
X = [(0.6..0.8),(0.6..0.8)]

#計算対象となる方程式を定義
f(x, y) = x^2 + y^2 - 1
g(x, y) = x - y
h( (x, y) ) = [f(x, y); g(x, y)]
# ForwardDiff.jacobian(g, X::IntervalBox) = ForwardDiff.jacobian(g, X.v)
#J=f'(I)

#区間Xにおけるヤコビ行列を計算
J = ForwardDiff.jacobian(h,X)

2×2 Array{Interval{Float64},2}:
     [1.19999, 1.60001]        [1.19999, 1.60001]
 [1, 1]                  [-1, -1]

In [50]:
#mid(X)を定義
C = map(Interval,X)
c1 = mid(C[1])
c2 = mid(C[2])

c = [c1,c2]
#Rを計算するためのf'(c)を求める
T = ForwardDiff.jacobian(h,c)

2×2 Array{Float64,2}:
 1.4   1.4
 1.0  -1.0

In [43]:
#区間Xにおけるヤコビ行列Jの逆行列Rを定義する
R = inv(T)

2×2 Array{Float64,2}:
 0.357143   0.5
 0.357143  -0.5

In [44]:
#単位行列を生成
Matrix{Float64}(I,2,2)

M = Matrix{Float64}(I,2,2) - R*J

2×2 Array{Interval{Float64},2}:
 [-0.0714286, 0.0714286]  [-0.0714286, 0.0714286]
 [-0.0714286, 0.0714286]  [-0.0714286, 0.0714286]

In [45]:
#Krawczyk写像を計算
K = c - R*h(c) + M*(X - c)

2-element Array{Interval{Float64},1}:
 [0.692857, 0.721429]
 [0.692857, 0.721429]

In [49]:
#収束判定
if 
    println('The exact solution is enclosed in')
    println(X)
else
    println('Krawczyk test is failed')
end

LoadError: syntax: invalid character literal

以上の手順によって、Krawczyk法が実装される

### 参考文献

1. 大石進一編著, 精度保証付き数値計算の基礎, コロナ社, 2018.<br>
(精度保証付き数値計算の教科書. 浮動小数点数および区間演算に詳しい. 今回は6章を参考にした)
1. 非線形方程式の解の精度保証 (+ 自動微分)<br>
http://www.kashi.info.waseda.ac.jp/~kashi/lec2020/nac/krawczyk.pdf